In [ ]:
# install chromium, its driver, and selenium
!apt -qq update
!apt -qq install chromium-chromedriver
!pip -qq install selenium
# !pip install -qq python-pptx

In [4]:
%cd /content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2

from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import os
import glob
import calendar

import sys
sys.path.append('01_utils')
# from getdata import *
# from processdata import *
from hlpr import *
from config import *

import warnings
warnings.filterwarnings('ignore')

/content/drive/My Drive/Colab Notebooks/AMEX/ResponseEngineDataUpdate_v2


In [5]:
# move to hlpr
def isnan(value):
  try:
    return np.isnan(value)
  except:
    return False

def skip_rows(df):
  '''
  Find the first row that is non nan witin the first column
  '''
  cnt = 0
  for i in df.iloc[:, 0]:
    if not(isnan(i)):
      break
    cnt += 1
  return cnt

def to_date(source, list_of_dates):
  '''
  Given the source, convert date str to datetime object
  '''
  if source == 'bls_uempl':
    month_to_num = {month: index for index, month in enumerate(calendar.month_abbr) if month}
    list_of_dates = [l for l in list_of_dates if type(l)==str]
    abbr_dict = {d: datetime(int('20' + d.split('-')[-1][:2]), month_to_num[d.split('-')[0]], 1) for d in list_of_dates}

    return abbr_dict

In [6]:
mastermap_ = mastermap()

In [64]:
# import data
filepath = os.path.join(DATA_PATH, 'uempl')
dfs = [pd.read_excel(f) for f in glob.glob(os.path.join(filepath, '*.xlsx'))]
df = pd.concat(dfs)

# find true row start and adjust
skip = skip_rows(df)
new_col = [c.lower() for c in list(df.iloc[skip, :].values)]
rename_col = {c:'uempl_rate' for c in new_col if '%' in c}
df.columns = new_col
df = df.iloc[skip+1:, :]
df.rename(columns=rename_col, inplace=True)

# remove str from rate
remove_from_rate = [i for i in df['uempl_rate'].unique() if type(i)==str]
df = df[-df['uempl_rate'].isin(remove_from_rate)]
df['uempl_rate'] = pd.to_numeric(df['uempl_rate'])/100

# rename cols
cols = ['laus code', 'state_fips_code', 'county_fips_code', 'county name/state abbreviation', 'period', 'force', 'employed', 'unemployed', 'uempl_rate']
df.columns = cols

# convert period to datetime obj
date_dict = to_date('bls_uempl', list(df['period'].unique()))
df['month'] = df['period'].apply(lambda x: date_dict[x] if x in date_dict.keys() else None)
df = df[-df['month'].isna()]
list_of_months = list(df[['month']].drop_duplicates().sort_values(by='month')['month'])

# grab the last 6-months
# max_month = list_of_months[-1]
# min_month = max_month - timedelta(days=183)
# df = df[df['month'] >= min_month]

# get subset of last 2-months of data
df_recent_rates = df[df['month'].isin(list_of_months[-2:])][['laus code', 'month', 'uempl_rate', 'force', 'employed', 'unemployed']]
df_recent_rates.sort_values(by=['laus code', 'month'], inplace=True)

# calculate mom
metrics = ['uempl_rate', 'force', 'employed', 'unemployed']
for m in metrics:
  df_recent_rates[f'{m}_mom'] = df_recent_rates.groupby('laus code')[m].transform(lambda x: x.diff())
  df_recent_rates[f'{m}_mom_%'] = df_recent_rates.groupby('laus code')[m].transform(lambda x: x.pct_change())
df_recent_rates = df_recent_rates[df_recent_rates['month']==max_month]
df_recent_rates.drop(columns=['month'] + metrics, inplace=True)

# combine to df
df = df.merge(df_recent_rates, how='left', on=['laus code'])
df = df[df['month'] >= max_month]

# get the fips
df['fips_text'] = df.apply(lambda x: '_'+x['state_fips_code']+x['county_fips_code'], axis=1)
df = df.merge(mastermap_['county_pop'], how='left', on='fips_text')
df = df[-df['fips'].isna()]

# filter
pop_col = [c for c in df.columns if 'popestimate' in c.lower()][0]
fltr_cols = ['market_name', 'county', 'fips', 'month', 
             'force', 'employed', 'unemployed', 'uempl_rate', 
             'force_mom', 'employed_mom', 'unemployed_mom', 'uempl_rate_mom', 
             'force_mom_%', 'employed_mom_%', 'unemployed_mom_%', 'uempl_rate_mom_%', pop_col]
df = df[fltr_cols]
df.rename(columns={pop_col:'population'}, inplace=True)